Copyright (c) MONAI Consortium  
Licensed under the Apache License, Version 2.0 (the "License");  
you may not use this file except in compliance with the License.  
You may obtain a copy of the License at  
&nbsp;&nbsp;&nbsp;&nbsp;http://www.apache.org/licenses/LICENSE-2.0  
Unless required by applicable law or agreed to in writing, software  
distributed under the License is distributed on an "AS IS" BASIS,  
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
See the License for the specific language governing permissions and  
limitations under the License.

## Setup environment

In [1]:
!python -c "import monai" || pip install -q "monai-weekly[ignite,pyyaml]"

## Setup imports

In [2]:
from monai.config import print_config

print_config()

MONAI version: 1.3.dev2340
Numpy version: 1.26.0
Pytorch version: 2.0.1+cu117
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 8d89083eeb8005babd7b5f76df83c1c80276cc10
MONAI __file__: /home/<username>/miniconda3/envs/monai_tutorial/lib/python3.9/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.11
ITK version: 5.3.0
Nibabel version: 5.1.0
scikit-image version: 0.21.0
scipy version: 1.11.3
Pillow version: 10.0.1
Tensorboard version: 2.14.1
gdown version: 4.7.1
TorchVision version: 0.15.2+cu117
tqdm version: 4.66.1
lmdb version: 1.4.1
psutil version: 5.9.0
pandas version: 2.1.1
einops version: 0.7.0
transformers version: 4.21.3
mlflow version: 2.7.1
pynrrd version: 1.0.0
clearml version: 1.13.1

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies


# Spleen Segmentation Lightning Bundle

In this tutorial we'll describe how to create a bundle for a segmentation network. This will include how to train and apply the network on the command line. Medical  will be used as the dataset with the bundle based off the [Spleen 3D segmentation with MONAI](https://github.com/Project-MONAI/tutorials/blob/main/3d_segmentation/spleen_segmentation_3d_lightning.ipynb) from Spleen segmentation using Task_09 subset from the Medical Segmentation Decathlon.

This work is licensed under the Creative Commons Attribution-ShareAlike 4.0 International License. To view a copy of this license, visit http://creativecommons.org/licenses/by-sa/4.0/.


You can run this cell or save it to a file and run it on the command line. A `DenseNet` based network will be trained to classify MedNIST images into one of six categories. Mostly this script uses Ignite-based classes such as `SupervisedTrainer` which is great for converting into a bundle. Let's start by initialising a bundle directory structure:

In [3]:
%%bash

python -m monai.bundle init_bundle SpleenSegLightning
which tree && tree SpleenSegLightning || true

## Metadata

We'll first replace the `metadata.json` file with our description of what the network will do:

In [90]:
%%writefile SpleenSegLightning/configs/metadata.json

{
    "version": "0.0.1",
    "changelog": {
        "0.0.1": "Initial version"
    },
    "monai_version": "1.2.0",
    "pytorch_version": "2.0.0",
    "numpy_version": "1.23.5",
    "optional_packages_version": {},
    "name": "SpleenSegLightning",
    "task": "3D Spleen segmentation network using MONAI and Pytorch Lightning",
    "description": "This is a demo network for segmentation of the spleen from 3D MRI images.",
    "authors": "Oeslle Lucena",
    "copyright": "Copyright (c) Oeslle Lucena",
    "data_source": "Task_09 subset from the Medical Segmentation Decathlon",
    "data_type": "Nifti",
    "intended_use": "This is suitable for demonstration only",
    "network_data_format":{
        "inputs": {
            "image": {
                "type": "image",
                "format": "magnitude",
                "modality": "MR",
                "num_channels": 1,
                "spatial_shape": [160, 160, 160],
                "dtype": "float32",
                "value_range": [0, 1],
                "is_patch_data": false,
                "channel_def": {"0": "image"}
            }
        },
        "outputs":{
            "pred": {
                "type": "image",
                "format": "labels",
                "num_channels": 2,
                "spatial_shape": [160, 160, 160],
                "dtype": "float32",
                "value_range": [],
                "is_patch_data": false,
                "channel_def": {"0": "background", "1": "spleen"}
            }
        }
    }
}

Overwriting SpleenSegLightning/configs/metadata.json


This contains more information compared to the previous tutorial's file. For inputs the network, a tensor "image" is given as a 64x64 sized single-channel image. This is one of the MedNIST images whose modality varies but will have a value range of `[0, 1]` after rescaling in the transform pipeline. The channel definition states the meaning of each channel, this input has only one which is the greyscale image itself. For network outputs there is only one, "pred", representing the prediction of the network as a tensor of size 6. Each of the six values is a prediction of that class which is described in `channel_def`.

## Common Definitions

What we'll now do is construct the bundle configuration scripts to implement training, testing, and inference based off the original script file given above. Common definitions should be placed in a common file used with other scripts to reduce duplication. In our original script, the network definition and transform sequence will be used in multiple places so should go in this common file:

In [145]:
%%writefile SpleenSegLightning/configs/common.yaml

# common imports
imports: 
- $import glob
- $import os

# define a default root directory value, this can overridden on the command line
bundle_dir: .
data_dir: $@bundle_dir+"/Task09_Spleen"

# define hyperparameters for the lightning trainer
max_epochs: 2
default_root_dir: $@bundle_dir+"/lightning_logs"
check_val_every_n_epoch: 1

lightninig_param:  '${
    ''max_epochs'': @max_epochs,
    ''default_root_dir'': @default_root_dir,
    ''check_val_every_n_epoch'': @check_val_every_n_epoch,
}'

# define a train and validation files from the data directory
train_images: '$sorted(glob.glob(os.path.join(@data_dir, ''imagesTr'', ''*.nii.gz'')))'
train_labels: '$sorted(glob.glob(os.path.join(@data_dir, ''labelsTr'', ''*.nii.gz'')))'

data_dicts: '$[{''image'': img, ''label'': lbl} for img, lbl in zip(@train_images, @train_labels)]'
train_files: '$@data_dicts[:-9]'
val_files: '$@data_dicts[-9:]'

    

Overwriting SpleenSegLightning/configs/common.yaml


# Scripts for training and evaluation

We'll define the training and evaluation yaml files and scripts contained the Pytorch Lightning based network.
First we'll create a script directory and a `model.py` file to contain the network definition:

In [5]:
!mkdir SpleenSegLightning/scripts

mkdir: cannot create directory ‘SpleenSegLightning/scripts’: File exists


In [116]:
%%writefile SpleenSegLightning/scripts/model.py

import pytorch_lightning
from monai.utils import set_determinism
from monai.transforms import (
    AsDiscrete,
    Compose,
    EnsureType,
)
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.data import decollate_batch
import torch


class MySegNet(pytorch_lightning.LightningModule):
    def __init__(self):
        super().__init__()
        self._model = UNet(
            spatial_dims=3,
            in_channels=1,
            out_channels=2,
            channels=(16, 32, 64, 128, 256),
            strides=(2, 2, 2, 2),
            num_res_units=2,
            norm=Norm.BATCH,
        )
        self.learning_rate = 1e-4
        self.loss_function = DiceLoss(to_onehot_y=True, softmax=True)
        self.post_pred = Compose([EnsureType("tensor", device="cpu"),
                                  AsDiscrete(argmax=True, to_onehot=2)])
        self.post_label = Compose([EnsureType("tensor", device="cpu"),
                                   AsDiscrete(to_onehot=2)])
        self.dice_metric = DiceMetric(include_background=False, reduction="mean",
                                      get_not_nans=False)
        self.best_val_dice = 0
        self.best_val_epoch = 0
        self.validation_step_outputs = []


    def forward(self, x):
        return self._model(x)

    def configure_optimizers(self):
        print("configure_optimizers", self.learning_rate)
        optimizer = torch.optim.Adam(self._model.parameters(), self.learning_rate)
        return optimizer

    def training_step(self, batch, batch_idx):
        images, labels = batch["image"], batch["label"]
        output = self.forward(images)
        loss = self.loss_function(output, labels)
        tensorboard_logs = {"train_loss": loss.item()}
        return {"loss": loss, "log": tensorboard_logs}

    def validation_step(self, batch, batch_idx):
        images, labels = batch["image"], batch["label"]
        roi_size = (160, 160, 160)
        sw_batch_size = 4
        outputs = sliding_window_inference(images, roi_size, sw_batch_size, self.forward)
        loss = self.loss_function(outputs, labels)
        outputs = [self.post_pred(i) for i in decollate_batch(outputs)]
        labels = [self.post_label(i) for i in decollate_batch(labels)]
        self.dice_metric(y_pred=outputs, y=labels)
        d = {"val_loss": loss, "val_number": len(outputs)}
        self.validation_step_outputs.append(d)
        return d

    def on_validation_epoch_end(self):
        val_loss, num_items = 0, 0
        for output in self.validation_step_outputs:
            val_loss += output["val_loss"].sum().item()
            num_items += output["val_number"]
        mean_val_dice = self.dice_metric.aggregate().item()
        self.dice_metric.reset()
        mean_val_loss = torch.tensor(val_loss / num_items)
        tensorboard_logs = {
            "val_dice": mean_val_dice,
            "val_loss": mean_val_loss,
        }
        if mean_val_dice > self.best_val_dice:
            self.best_val_dice = mean_val_dice
            self.best_val_epoch = self.current_epoch
        print(
            f"current epoch: {self.current_epoch} "
            f"current mean dice: {mean_val_dice:.4f}"
            f"\nbest mean dice: {self.best_val_dice:.4f} "
            f"at epoch: {self.best_val_epoch}"
        )
        self.validation_step_outputs.clear()  # free memory
        return {"log": tensorboard_logs}




Overwriting SpleenSegLightning/scripts/model.py


Next, we'll create a `main.py` file to contain the training and evaluation scripts:

In [132]:
%%writefile SpleenSegLightning/scripts/main.py

from scripts.model import MySegNet
import pytorch_lightning


def train(lightninig_param, train_dl, val_dl):
    net = MySegNet()
    trainer = pytorch_lightning.Trainer(lightninig_param['max_epochs'], 
                                        lightninig_param['default_root_dir'],
                                        lightninig_param['check_val_every_n_epoch'])
    trainer.fit(model=net, train_dataloaders=train_dl, val_dataloaders=val_dl)

def evaluate(ckpt_file, val_dl):
    net = MySegNet().load_from_checkpoint(ckpt_file)
    trainer = pytorch_lightning.Trainer(devices=1, num_nodes=1)
    trainer.validate(model=net, dataloaders=val_dl)

Overwriting SpleenSegLightning/scripts/main.py


## Training
Now, we'll define a `train.yaml` file to be used to set the configurations for the training stage:


In [88]:
%%writefile SpleenSegLightning/configs/train.yaml

imports:
- $import glob
- $import os
- $from scripts.main import train

# define a default root directory value, this can overridden on the command line
bundle_dir: .
data_dir: $@bundle_dir+"/Task09_Spleen"

# define hyperparameters for the lightning trainer
max_epochs: 2
default_root_dir: $@bundle_dir+"/lightning_logs"
check_val_every_n_epoch: 1

lightninig_param:  '${
    ''max_epochs'': @max_epochs,
    ''default_root_dir'': @default_root_dir,
    ''check_val_every_n_epoch'': @check_val_every_n_epoch,
}'

# define a train and validation files from the data directory
train_images: '$sorted(glob.glob(os.path.join(@data_dir, ''imagesTr'', ''*.nii.gz'')))'
train_labels: '$sorted(glob.glob(os.path.join(@data_dir, ''labelsTr'', ''*.nii.gz'')))'

data_dicts: '$[{''image'': img, ''label'': lbl} for img, lbl in zip(@train_images, @train_labels)]'
train_files: '$@data_dicts[:-9]'
val_files: '$@data_dicts[-9:]'


# define a transform sequence by instantiating a Compose instance with a transform sequence
train_transform:
  _target_: Compose
  transforms:
  - _target_: LoadImaged
    keys: ['image','label']
    image_only: true
  - _target_: EnsureChannelFirstd
    keys: ['image','label']
  - _target_: Orientationd
    keys: ['image','label']
    axcodes: 'RAS'
  - _target_: Spacingd
    keys: ['image','label']
    pixdim: [1.5, 1.5, 2.0]
  - _target_: ScaleIntensityRanged
    keys: ['image']
    a_min: -57
    a_max: 164
    b_min: 0.0
    b_max: 1.0
    clip: True
  - _target_: CropForegroundd
    keys: ['image','label']
    source_key: 'image'
  - _target_: RandCropByPosNegLabeld
    keys: ['image','label']
    label_key: 'label'
    spatial_size: [96, 96, 96]
    pos: 1
    neg: 1
    num_samples: 4
    image_key: 'image'
    image_threshold: 0

val_transform:
  _target_: Compose
  transforms:
  - _target_: LoadImaged
    keys: ['image','label']
    image_only: true
  - _target_: EnsureChannelFirstd
    keys: ['image','label']
  - _target_: Orientationd
    keys: ['image','label']
    axcodes: 'RAS'
  - _target_: Spacingd
    keys: ['image','label']
    pixdim: [1.5, 1.5, 2.0]
  - _target_: ScaleIntensityRanged
    keys: ['image']
    a_min: -57
    a_max: 164
    b_min: 0.0
    b_max: 1.0
    clip: True
  - _target_: CropForegroundd
    keys: ['image','label']
    source_key: 'image'

val_dataset:
  _target_: CacheDataset
  data: '@val_files'
  transform: '@val_transform'
  cache_rate: 1.0
  num_workers: 4

train_dataset:
  _target_: CacheDataset
  data: '@train_files'
  transform: '@train_transform'
  cache_rate: 1.0
  num_workers: 4
  
train_dl:
  _target_: DataLoader
  dataset: '@train_dataset'
  batch_size: 1
  shuffle: true
  num_workers: 4
  
val_dl:
  _target_: DataLoader
  dataset: '@val_dataset'
  batch_size: 1
  shuffle: false
  num_workers: 4

train:
- '$train(@lightninig_param, @train_dl, @val_dl)'

Overwriting SpleenSegLightning/configs/train.yaml


There is a lot going on here but hopefully you see how this replicates the object definitions in the original source file. A few specific points:
* References are made to objects defined in `common.yaml` such as `@root_dir`, so this file needs to be used in conjunction with this one.
* A `max_epochs` hyperparameter is provided whose value you can change on the command line, eg. `--max_epochs 5`.
* Definitions for the `optimizer`, `loss_function`, and `inferer` arguments of `trainer` are provided inline but it would be better practice to define these separately.
* The learning rate is hard-coded as `1e-5`, it would again be better practice to define a separate `lr` hyperparameter, although it can be changed on the command line with `'--trainer#optimizer#lr' 0.001`.
* The trained network is saved using Pytorch's `jit` module directly, better practice would be to provide a handler, such as `CheckpointSaver`, to the trainer or to an evaluator object, see other tutorial examples on how to do this. This was kept here to match the original example.

Now the network can be trained by running the bundle:

In [89]:
%%bash

BUNDLE="./SpleenSegLightning"

export PYTHONPATH="$BUNDLE"

# run the bundle with epochs set to 2 for speed during testing, change this to get a better result
python -m monai.bundle run train \
    --bundle_dir "$BUNDLE" \
    --meta_file "$BUNDLE/configs/metadata.json" \
    --config_file "$BUNDLE/configs/train.yaml" \
    --max_epochs 2

workflow_name None
config_file ./SpleenSegLightning/configs/train.yaml
meta_file None
logging_file None
init_id None
run_id train
final_id None
tracking None
bundle_dir ./SpleenSegLightning
max_epochs 2
2023-10-09 14:36:53,640 - INFO - --- input summary of monai.bundle.scripts.run ---
2023-10-09 14:36:53,640 - INFO - > config_file: './SpleenSegLightning/configs/train.yaml'
2023-10-09 14:36:53,640 - INFO - > run_id: 'train'
2023-10-09 14:36:53,640 - INFO - > bundle_dir: './SpleenSegLightning'
2023-10-09 14:36:53,641 - INFO - > max_epochs: 2
2023-10-09 14:36:53,641 - INFO - ---


monai.bundle.workflows ConfigWorkflow.__init__:workflow_type: Current default value of argument `workflow_type=None` has been deprecated since version 1.2. It will be changed to `workflow_type=train` in version 1.4.
Default logging file in SpleenSegLightning/configs/logging.conf does not exist, skipping logging.
monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.
Loading dataset: 100%|██████████| 9/9 [00:11<00:00,  1.30s/it]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/2


workflow_name None
config_file ./SpleenSegLightning/configs/train.yaml
meta_file None
logging_file None
init_id None
run_id train
final_id None
tracking None
bundle_dir ./SpleenSegLightning
max_epochs 2
2023-10-09 14:37:57,731 - INFO - --- input summary of monai.bundle.scripts.run ---
2023-10-09 14:37:57,732 - INFO - > config_file: './SpleenSegLightning/configs/train.yaml'
2023-10-09 14:37:57,732 - INFO - > run_id: 'train'
2023-10-09 14:37:57,732 - INFO - > bundle_dir: './SpleenSegLightning'
2023-10-09 14:37:57,732 - INFO - > max_epochs: 2
2023-10-09 14:37:57,732 - INFO - ---


monai.bundle.workflows ConfigWorkflow.__init__:workflow_type: Current default value of argument `workflow_type=None` has been deprecated since version 1.2. It will be changed to `workflow_type=train` in version 1.4.
Default logging file in SpleenSegLightning/configs/logging.conf does not exist, skipping logging.
monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.
Loading dataset: 100%|██████████| 9/9 [00:09<00:00,  1.09s/it]
Initializing distributed: GLOBAL_RANK: 1, MEMBER: 2/2
----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distributed processes registered. Starting with 2 processes
----------------------------------------------------------------------------------------------------

LOCAL_RANK: 1 - CUDA_VISIBLE_DEVICES: [0,1]
LOC

configure_optimizers 0.0001
Sanity Checking: 0it [00:00, ?it/s]configure_optimizers 0.0001
Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:06<00:00,  3.44s/it]current epoch: 0 current mean dice: 0.0431
best mean dice: 0.0431 at epoch: 0


The number of training batches (16) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


                                                                           current epoch: 0 current mean dice: 0.0431
best mean dice: 0.0431 at epoch: 0
Epoch 0: 100%|██████████| 16/16 [00:19<00:00,  1.21s/it, v_num=7]
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0: 100%|██████████| 5/5 [00:10<00:00,  2.06s/it]current epoch: 0 current mean dice: 0.0196
best mean dice: 0.0431 at epoch: 0

Epoch 0: 100%|██████████| 16/16 [00:54<00:00,  3.39s/it, v_num=7]current epoch: 0 current mean dice: 0.0196
best mean dice: 0.0431 at epoch: 0
Epoch 1: 100%|██████████| 16/16 [00:18<00:00,  1.18s/it, v_num=7]
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0: 100%|██████████| 5/5 [00:10<00:00,  2.19s/it]current epoch: 1 current mean dice: 0.0357
best mean dice: 0.0431 at epoch: 0


`Trainer.fit` stopped: `max_epochs=2` reached.


Epoch 1: 100%|██████████| 16/16 [00:54<00:00,  3.42s/it, v_num=7]current epoch: 1 current mean dice: 0.0357
best mean dice: 0.0431 at epoch: 0


## Evaluation


The `scripts` directory has to be a valid Python module so needs a `__init__.py` file, you can include other files and import them separately or import their members into this file. Here we defined `evaluate` to enclose the loop from the original script. This can then be called as part of a expression sequence "program":

In [146]:
%%writefile SpleenSegLightning/configs/evaluate.yaml

# common imports
imports: 
- $from scripts.main import evaluate

ckpt_file: ""

val_transform:
  _target_: Compose
  transforms:
  - _target_: LoadImaged
    keys: ['image','label']
    image_only: true
  - _target_: EnsureChannelFirstd
    keys: ['image','label']
  - _target_: Orientationd
    keys: ['image','label']
    axcodes: 'RAS'
  - _target_: Spacingd
    keys: ['image','label']
    pixdim: [1.5, 1.5, 2.0]
  - _target_: ScaleIntensityRanged
    keys: ['image']
    a_min: -57
    a_max: 164
    b_min: 0.0
    b_max: 1.0
    clip: True
  - _target_: CropForegroundd
    keys: ['image','label']
    source_key: 'image'

val_dataset:
  _target_: CacheDataset
  data: '@val_files'
  transform: '@val_transform'
  cache_rate: 1.0
  num_workers: 4
  
val_dl:
  _target_: DataLoader
  dataset: '@val_dataset'
  batch_size: 1
  shuffle: false
  num_workers: 4
  
# loads the weights from the given file (which needs to be set on the command line) then calls "evaluate"
evaluate:
- '$evaluate(@ckpt_file, @val_dl)'


Overwriting SpleenSegLightning/configs/evaluate.yaml


Evaluation is then run on the command line, using "evaluate" as the program to run and providing a path to the model weights with the `ckpt_file` variable:

In [147]:
%%bash

BUNDLE="./SpleenSegLightning"
export PYTHONPATH="$BUNDLE"

python -m monai.bundle run evaluate \
    --meta_file "$BUNDLE/configs/metadata.json" \
    --config_file "['$BUNDLE/configs/common.yaml','$BUNDLE/configs/evaluate.yaml']" \
    --ckpt_file "$BUNDLE/models/epoch=599-step=9600.ckpt"

workflow_name None
config_file ['./SpleenSegLightning/configs/common.yaml', './SpleenSegLightning/configs/evaluate.yaml']
meta_file ./SpleenSegLightning/configs/metadata.json
logging_file None
init_id None
run_id evaluate
final_id None
tracking None
ckpt_file ./SpleenSegLightning/models/epoch=599-step=9600.ckpt
2023-10-09 15:51:06,913 - INFO - --- input summary of monai.bundle.scripts.run ---
2023-10-09 15:51:06,913 - INFO - > config_file: ['./SpleenSegLightning/configs/common.yaml',
 './SpleenSegLightning/configs/evaluate.yaml']
2023-10-09 15:51:06,913 - INFO - > meta_file: './SpleenSegLightning/configs/metadata.json'
2023-10-09 15:51:06,913 - INFO - > run_id: 'evaluate'
2023-10-09 15:51:06,913 - INFO - > ckpt_file: './SpleenSegLightning/models/epoch=599-step=9600.ckpt'
2023-10-09 15:51:06,913 - INFO - ---


monai.bundle.workflows ConfigWorkflow.__init__:workflow_type: Current default value of argument `workflow_type=None` has been deprecated since version 1.2. It will be changed to `workflow_type=train` in version 1.4.
Default logging file in SpleenSegLightning/configs/logging.conf does not exist, skipping logging.
Traceback (most recent call last):
  File "/home/ol18/miniconda3/envs/monai_tutorial/lib/python3.9/site-packages/monai/bundle/config_item.py", line 377, in evaluate
    return eval(value[len(self.prefix) :], globals_, locals)
  File "<string>", line 1, in <module>
NameError: name 'glob' is not defined

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/ol18/miniconda3/envs/monai_tutorial/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/ol18/miniconda3/envs/monai_tutorial/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_global

CalledProcessError: Command 'b'\nBUNDLE="./SpleenSegLightning"\nexport PYTHONPATH="$BUNDLE"\n\npython -m monai.bundle run evaluate \\\n    --meta_file "$BUNDLE/configs/metadata.json" \\\n    --config_file "[\'$BUNDLE/configs/common.yaml\',\'$BUNDLE/configs/evaluate.yaml\']" \\\n    --ckpt_file "$BUNDLE/models/epoch=599-step=9600.ckpt"\n'' returned non-zero exit status 1.

## Summary and Next

This tutorial has covered:
* Creating full training scripts in bundles
* Training a network then evaluating it's performance with scripts

That's it to creating a bundle to match an existing script. It was mentioned in a number of places that best practice wasn't followed to stick to the original script's structure, so further tutorials will cover this in greater detail. 